The Australian DEM is 35GB, and hosted by the DEA [here](https://data.dea.ga.gov.au/?prefix=projects/elevation/ga_srtm_dem1sv1_0/)
AWS bucket name to use to access the data is `dea-public-data`

## Original Data Source

*SRTM-derived 1 Second Digital Elevation Models Version 1.0*

The 1 second Shuttle Radar Topography Mission (SRTM) Digital Elevation
Models Version 1.0 package comprises three surface models: the Digital
Elevation Model (DEM), the Smoothed Digital Elevation Model (DEM-S)
and the Hydrologically Enforced Digital Elevation Model (DEM-H).

### sources and guides
[DEA guide to accessing data from their public S3 bucket using STAC](https://docs.dea.ga.gov.au/guides/setup/gis/stac/)

[Downloading and streaming data using STAC metadata](https://docs.dea.ga.gov.au/notebooks/How_to_guides/Downloading_data_with_STAC/)

In [1]:
%pip install rasterio rioxarray pystac_client geoviews hvplot datashader

Defaulting to user installation because normal site-packages is not writeable
  Using cached requests-2.31.0-py3-none-any.whl (62 kB)
  Attempting uninstall: requests
    Found existing installation: requests 2.28.1
    Uninstalling requests-2.28.1:
      Successfully uninstalled requests-2.28.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
geodata-harvester 1.1.1 requires requests==2.28.1, but you have requests 2.31.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [2]:

import os
import warnings
import numpy as np
import pandas as pd
import xarray as sr
import json

# packages for widgets and interactive map:
import io
import base64

#packages for geo-processing:
import geopandas as gpd
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.windows import from_bounds
import rioxarray

import pystac_client
import pyproj

import geoviews as gv
import holoviews as hv
import hvplot.xarray

In [3]:
def initialize_stac_client(stac_url):
    """
    Initialize and return a STAC client for a given STAC API URL.
    Parameters:
    - stac_url (str): The URL of the STAC API.
    Returns:
    - A pystac_client.Client object
    """
    client = pystac_client.Client.open(stac_url)
    return client


def query_stac_api(client, bbox, collections, start_date=None, end_date=None, limit=10):
    """
    Query a STAC API for items within a bounding box and date range for specific collections.
    Parameters:
    - client: The STAC client initialized with `initialize_stac_client`.
    - bbox (list): The bounding box for the query [min_lon, min_lat, max_lon, max_lat].
    - collections (list): A list of collection IDs to include in the query.
    - start_date (str, optional): The start date for the query (YYYY-MM-DD). Defaults to None.
    - end_date (str, optional): The end date for the query (YYYY-MM-DD). Defaults to None.
    - limit (int): Maximum number of items to return.    
    Returns:
    - A list of STAC Items that match the query parameters.
    """

    search_params = {
        "bbox": bbox,
        "collections": collections,
        "limit": limit
    }
    if start_date and end_date:
        search_params["datetime"] = f"{start_date}/{end_date}"

    search = client.search(**search_params)

    items = list(search.items())
    return items


def inspect_stac_item(item):
    """
    Inspects a STAC item and prints out key information to help identify the data type.

    This function iterates over the assets of a given STAC item, printing details such as the
    asset's title, description, media type, and roles. These details can provide insights into
    the nature of the data contained within the item, such as whether it includes satellite imagery,
    elevation data, or other geospatial resources.

    Parameters:
    - item: A STAC item object. This object should conform to the STAC item specification and
            include properties like an ID, datetime, and a dictionary of assets.

    Returns:
    - None: This function does not return any value. It only prints information to the console.
    """

    # Print the unique identifier of the STAC item
    print("Item ID:", item.id)

    # Print the acquisition date of the data, which is stored in the item's properties
    print("Date:", item.properties.get('datetime'))

    # Begin iterating over the assets associated with this STAC item.
    # Assets represent individual data files or resources related to this item.
    print("Assets:")
    for asset_key, asset in item.assets.items():
        # asset_key is the name used to refer to this asset in the STAC item's assets dictionary.
        # asset is the actual asset object, which contains metadata about the data file or resource.

        # Print the key of the asset and its title. The title is a human-readable name for the asset.
        # If no title is provided, it defaults to 'No title'.
        print(f"  - {asset_key}: {asset.title or 'No title'}")

        # Print a description of the asset, which can provide more context about the data it contains.
        # If no description is provided, it defaults to 'No description'.
        print(f"    Description: {asset.description or 'No description'}")

        # Print the media type of the asset, which indicates the format of the data file (e.g., 'image/tiff' for a GeoTIFF file).
        print(f"    Media Type: {asset.media_type}")

        # Print the roles associated with this asset. Roles are used to describe the function of the asset,
        # such as whether it's the primary data ('data'), metadata about the item ('metadata'), a thumbnail image ('thumbnail'), etc.
        # The roles are joined by a comma in case there are multiple roles.
        print(f"    Roles: {', '.join(asset.roles)}")

## environment and STAC setup

In [4]:
os.environ['AWS_NO_SIGN_REQUEST'] = 'YES'

"""
If STAC service is down, try using direct link to the DEM-H geotiff file
"""
dem_href = 'https://dea-public-data.s3-ap-southeast-2.amazonaws.com/projects/elevation/ga_srtm_dem1sv1_0/dem1sv1_0.tif'

# set the catalog and collection to interact with
CATALOG_DEM = 'https://explorer.sandbox.dea.ga.gov.au/stac/'
COLLECTION_DEM = ['ga_srtm_dem1sv1_0']

#set the area of interest (aoi), in this case a geojson polygon
BOUNDARIES = 'input-data/dissolved-boundaries.geojson'
geom = gpd.read_file(BOUNDARIES)
bbox = list(geom.total_bounds)


crs_string = "epsg:3857"
epsg = pyproj.CRS.from_string(crs_string).to_epsg()
resolution = 90 #added to resolve error "Failed to auto-guess CRS/resolution."

#initialise the STAC client using the above catalog
catalog = initialize_stac_client(CATALOG_DEM)

bbox

[116.26012130269045,
 -29.307384715430175,
 116.3875862387774,
 -29.220237788279107]

## using uploaded aoi to get DEM

Download the fetched DEM raster:

In [5]:
stac_items = query_stac_api(catalog, bbox, COLLECTION_DEM, None, None, None)

# Only want the hydrologically-enforced dem asset
item = stac_items[0]
dem_asset = item.assets.get('dem_h')

dem_asset

<Asset href=s3://dea-public-data/projects/elevation/ga_srtm_dem1sv1_0/demh1sv1_0.tif>

In [6]:
# Take the DEM and save out a windowed geotif of the aoi
with rasterio.open(dem_asset.href) as src:
    # Convert the GeoJSON bounds to a rasterio window
    window = from_bounds(*bbox, transform=src.transform)

    # Read the data within the window
    data = src.read(window=window)

    # Preserve the metadata from the dem asset
    kwargs = src.meta.copy()
    # Updating the metadata based on the bounding box of our original geojson
    kwargs.update({
        'height': window.height,
        'width': window.width,
        'transform': rasterio.windows.transform(window, src.transform)
    })
    
    output_tiff = 'output-data/dem-h-elevation.tiff'
    with rasterio.open(output_tiff, 'w', **kwargs) as dst:
        dst.write(data)

In [7]:
img = gv.util.from_xarray(rioxarray.open_rasterio(output_tiff).rio.reproject('EPSG:3857'))
#gv.tile_sources.OSM * img.opts(cmap='viridis')
map_tiles = hv.element.tiles.EsriImagery().opts(width=1000, height=600)
map_img = hv.Image(img, kdims=['x','y']).opts(cmap='viridis')
map_combo = map_tiles * map_img
map_combo

:Overlay
   .Tiles.I :Tiles   [x,y]
   .Image.I :Image   [x,y]   (z)